# Machine Learning Model
En este apartado se hará el codigo correspondiente para la creación del modelo del sistema de recomendación. Para esto utilizaremos la métrica de similitud del coseno, ya que es una métrica simple, eficiente y robusta  para evaluar similitud entre dos vectrores en PLN, esta métrica es adecuada para datos númericos y también robusta con la frecuencia de palabras. 
Para aplicar la métrica de similitud del coseno debemos importarla desde la librería de sklearn.


In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
dfMachineLearning = pd.read_parquet('DB Steam/machineLearning.parquet')
dfMachineLearning.head()

score  \
title        ! That Bastard Is Trying To Steal Our Gold !   
user_id                                                     
--000--                                               NaN   
--ace--                                               NaN   
--ionex--                                             NaN   
-2SV-vuLB-Kg                                          NaN   
-Azsael-                                              NaN   

                                                                              \
title        //N.P.P.D. RUSH//- The milk of Ultraviolet 0RBITALIS 10,000,000   
user_id                                                                        
--000--                                             NaN       NaN        NaN   
--ace--                                             NaN       NaN        NaN   
--ionex--                                           NaN       NaN        NaN   
-2SV-vuLB-Kg                                        NaN       NaN        NaN   
-Azsael-                                            NaN       NaN        NaN   

                                \
title        100% Orange Juice   
user_id                          
--000--                    NaN   
--ace--                    NaN   
--ionex--                  NaN   
-2SV-vuLB-Kg               NaN   
-Azsael-                   NaN   

                                                                             \
title        100% Orange Juice - Krila &amp; Kae Character Pack 1001 Spikes   
user_id                                                                       
--000--                                                     NaN         NaN   
--ace--                                                     NaN         NaN   
--ionex--                                                   NaN         NaN   
-2SV-vuLB-Kg                                                NaN         NaN   
-Azsael-                                                    NaN         NaN   

                                     \
title        12 Labours of Hercules   
user_id                               
--000--                         NaN   
--ace--                         NaN   
--ionex--                       NaN   
-2SV-vuLB-Kg                    NaN   
-Azsael-                        NaN   

                                                                             \
title        12 Labours of Hercules II: The Cretan Bull 12 is Better Than 6   
user_id                                                                       
--000--                                             NaN                 NaN   
--ace--                                             NaN                 NaN   
--ionex--                                           NaN                 NaN   
-2SV-vuLB-Kg                                        NaN                 NaN   
-Azsael-                                            NaN                 NaN   

              ...                                                \
title         ... liteCam Game: 100 FPS Game Capture nail'd  oO   
user_id       ...                                                 
--000--       ...                                NaN    NaN NaN   
--ace--       ...                                NaN    NaN NaN   
--ionex--     ...                                NaN    NaN NaN   
-2SV-vuLB-Kg  ...                                NaN    NaN NaN   
-Azsael-      ...                                NaN    NaN NaN   

                                                           \
title        planetarian ~the reverie of a little planet~   
user_id                                                     
--000--                                               NaN   
--ace--                                               NaN   
--ionex--                                             NaN   
-2SV-vuLB-Kg                                          NaN   
-Azsael-                                              NaN   

                                            

Antes de entrenar el modelo, es importante normalizar los valores del dataframe, para mejorar el rendimiento del modelo. 
Así que se noramlizarán los valores haciendo el escalado z (restando la media de las calificaciones de los usuarios y dividiendo eso por la diferencia entre el valor máximo y mínimo de las calificaciones); esto transforma los valores de la columna a una escala estándar con una media de 0 y una desviación estándar de 1.
Recordemos que x = columna individual del dataframe.

In [3]:
dfMachineLearning = dfMachineLearning.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
dfMachineLearning = dfMachineLearning.fillna(0)
dfMachineLearning = dfMachineLearning.T
dfMachineLearning = dfMachineLearning.loc[:, (dfMachineLearning != 0).any(axis=0)]
dfMachineLearning

user_id                                             -2SV-vuLB-Kg  -PRoSlayeR-  \
      title                                                                     
score ! That Bastard Is Trying To Steal Our Gold !           0.0          0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet             0.0          0.0   
      0RBITALIS                                              0.0          0.0   
      10,000,000                                             0.0          0.0   
      100% Orange Juice                                      0.0          0.0   
...                                                          ...          ...   
      sZone-Online                                           0.0          0.0   
      the static speaks my name                              0.0          0.0   
      theHunter Classic                                      0.0          0.0   
      theHunter: Primal                                      0.0          0.0   
      Астролорды: Облако Оорта                               0.0          0.0   

user_id                                             -SEVEN-  \
      title                                                   
score ! That Bastard Is Trying To Steal Our Gold !      0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet        0.0   
      0RBITALIS                                         0.0   
      10,000,000                                        0.0   
      100% Orange Juice                                 0.0   
...                                                     ...   
      sZone-Online                                      0.0   
      the static speaks my name                         0.0   
      theHunter Classic                                 0.0   
      theHunter: Primal                                 0.0   
      Астролорды: Облако Оорта                          0.0   

user_id                                             -_PussyDestroyer_-  0-3-0  \
      title                                                                     
score ! That Bastard Is Trying To Steal Our Gold !                 0.0    0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                   0.0    0.0   
      0RBITALIS                                                    0.0    0.0   
      10,000,000                                                   0.0    0.0   
      100% Orange Juice                                            0.0    0.0   
...                                                                ...    ...   
      sZone-Online                                                 0.0    0.0   
      the static speaks my name                                    0.0    0.0   
      theHunter Classic                                            0.0    0.0   
      theHunter: Primal                                            0.0    0.0   
      Астролорды: Облако Оорта                                     0.0    0.0   

user_id                                             00000000000000000001227  \
      title                                                                   
score ! That Bastard Is Trying To Steal Our Gold !                      0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                        0.0   
      0RBITALIS                                                         0.0   
      10,000,000                                                        0.0   
      100% Orange Juice                                                 0.0   
...                                                                     ...   
      sZone-Online                                                      0.0   
      the static speaks my name                                         0.0   
      theHunter Classic                                                 0.0   
      theHunter: Primal                                                 0.0   
      Астролорды: Облако Оорта                                          0.0   

user_id                                     

In [4]:
# Hacemos la matriz dispersa, para no tomar los valores iguales a 0 y optimizar nuestro espacio en memoria.
dfMachineLearning_Sparse = sp.sparse.csr_matrix(dfMachineLearning.values)
dfMachineLearning_Sparse

<3122x6916 sparse matrix of type '<class 'numpy.float64'>'
	with 26778 stored elements in Compressed Sparse Row format>

Para determinar que tan parecidos son los juegos, basados en las recomendaciones de los usuarios, utilizamos la similitud del coseno, para este caso hacemos la matriz de similitud para evaluar la similitud de los dos vectores.

In [5]:
itemSimilar = cosine_similarity(dfMachineLearning_Sparse)

In [6]:
dfItemSim = pd.DataFrame(itemSimilar, index = dfMachineLearning.index, columns = dfMachineLearning.index)
dfItemSim.head()

score  \
title                                              ! That Bastard Is Trying To Steal Our Gold !   
      title                                                                                       
score ! That Bastard Is Trying To Steal Our Gold !                                          1.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                                            0.0   
      0RBITALIS                                                                             0.0   
      10,000,000                                                                            0.0   
      100% Orange Juice                                                                     0.0   

                                                                                               \
title                                              //N.P.P.D. RUSH//- The milk of Ultraviolet   
      title                                                                                     
score ! That Bastard Is Trying To Steal Our Gold !                                        0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                                          1.0   
      0RBITALIS                                                                           0.0   
      10,000,000                                                                          0.0   
      100% Orange Juice                                                                   0.0   

                                                                         \
title                                              0RBITALIS 10,000,000   
      title                                                               
score ! That Bastard Is Trying To Steal Our Gold !       0.0        0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet         0.0        0.0   
      0RBITALIS                                          1.0        0.0   
      10,000,000                                         0.0        1.0   
      100% Orange Juice                                  0.0        0.0   

                                                                      \
title                                              100% Orange Juice   
      title                                                            
score ! That Bastard Is Trying To Steal Our Gold !               0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                 0.0   
      0RBITALIS                                                  0.0   
      10,000,000                                                 0.0   
      100% Orange Juice                                          1.0   

                                                                                                       \
title                                              100% Orange Juice - Krila &amp; Kae Character Pack   
      title                                                                                             
score ! That Bastard Is Trying To Steal Our Gold !                                                0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet                                                  0.0   
      0RBITALIS                                                                                   0.0   
      10,000,000                                                                                  0.0   
      100% Orange Juice                                                                           0.0   

                                                                \
title                                              1001 Spikes   
      title                                                      
score ! That Bastard Is Trying To Steal Our Gold !         0.0   
      //N.P.P.D. RUSH//- The milk of Ultraviolet           0.0   
      0RBITALIS                                            0.0   
      10,000,000                                           0.0   
      100% Orange Juice                

In [7]:
dfItemSim.to_parquet('DB Steam/item_Sim.parquet')

El sistema funciona de la siguiente manera: Se ingresa el nombre de un juego como input, seguidamente el sistema ordena los valores de la columna correspondiente a ese juego de forma descendente en la matriz de similitud entre items, de esa manera se cerciora que los juegos más similares ocupen los primeros puestos del orden. Luego, selecciona los 5 juegos más similares y los lista. 